In [1]:
from pyspark.sql.functions import col

# ============================================================
# 0. Configurações de Otimização do Fabric
# ============================================================
# Habilita o V-Order (padrão no Fabric, mas garante escrita otimizada)
spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.binSize", "1073741824") # ~1GB por arquivo se possível

# ============================================================
# 1. Caminho físico
# ============================================================
path_destino = (
    "abfss://ws_departamento_pessoal@onelake.dfs.fabric.microsoft.com/"
    "lk_departamento_pessoal.Lakehouse/Tables/tab_gold_dim_funcionario_ultima_posicao_mes"
)

# ============================================================
# 2. Ler e Filtrar
# ============================================================
df_historico = spark.table("tab_gold_dim_funcionario_historico")

df_filtrado = df_historico.filter(col("RK_MES") == 1)

# ============================================================
# 3. CORREÇÃO DO SKEW
#
#    Removido col("CODCOLIGADA"), col("SECAO").
#    Usando apenas o número, o Spark faz um Round-Robin,
#    distribuindo os dados igualmente independente do conteúdo.
# ============================================================
NUM_PARTICOES = 200  # Se tiver menos de 10GB de dados, pode até baixar para 20-50

df_reparticionado = df_filtrado.repartition(NUM_PARTICOES)

# ============================================================
# 4. Escrever
# ============================================================
(
    df_reparticionado.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true") # Boa prática se a estrutura mudar
        .save(path_destino)
)

print(f"Gravação concluída. Partições utilizadas: {NUM_PARTICOES}")

StatementMeta(, cb0f5827-d535-4e18-817c-d3c05d6156cc, 3, Finished, Available, Finished)